In [1]:
import jax
import jax.numpy as jnp
import jax_md
import numpy as np

from ase.io import read
from jax_md import units
from typing import Dict

from sup_gems import to_jax_md
from sup_gems import SupGemsPotential

import time

In [2]:
# Some helper functions used throughout the example notebook.

# Default nose hoover chain parameters.
def default_nhc_kwargs(
    tau: jnp.float32, 
    overrides: Dict
) -> Dict:
    
    default_kwargs = {
        'chain_length': 3, 
        'chain_steps': 2, 
        'sy_steps': 3,
        'tau': tau
    }
    
    if overrides is None:
        return default_kwargs
  
    return {
      k: overrides.get(k, default_kwargs[k]) for k in default_kwargs
    }

# Read the Molecule

Start by reading the molecular structure. Here we assume some ASE digestable file, e.g. and `XYZ` file.

In [3]:
# Read some molecular structure. Here we read the DHA structure from the test directory.
path_to_xyz = '../tests/test_data/dha.xyz'
atoms = read(path_to_xyz, index=-1)

positions_init = jnp.array(atoms.get_positions())
species = jnp.array(atoms.get_atomic_numbers())
masses = jnp.array(atoms.get_masses())

# Prepare JAX-MD Energy Function

JAX-MD is build around the energy function and the neighborlist. Since we have short- and long-range neighbors, we have two neighborlists compared to the standard case of only a single one. We use the convenience interface `to_jax_md` provided within this repository, which returns the SO3LR energy function as well as the two neighborlists. It takes as input the `SupGemsPotential` itself, as well as a displacement function which determines how update the atomic positions. The `displacement` function calculates the displacement vectors between atoms and can differ, e.g. based on the fact if simulations are performed under perdioic boundary conditions (PBCs) or not. Also positions can be either represented in real or fractional coordinates. For more details on displacement functions as well as on fractional coordinates we refer to the JAX-MD docs. Here we simulate DHA in vacuum (no PBCs) which allows to use `jax_md.space.free` to obtain an displacement function. It also returns a `shift` function which determines how to update the atomic positions during simulation.

In [4]:
displacement, shift = jax_md.space.free()
box = None  # No simulation box. 

neighbor_fn, neighbor_fn_lr, energy_fn = to_jax_md(
    potential=SupGemsPotential(),
    displacement_or_metric=displacement,
    box_size=box,
    species=species,
    capacity_multiplier=1.25,
    buffer_size_multiplier_sr=1.25,
    buffer_size_multiplier_lr=1.25,
    minimum_cell_size_multiplier_sr=1.0,
    disable_cell_list=True,  # Cell list partitioning can only be applied if there is a simulation box.
    fractional_coordinates=False
)

# Energy function.
energy_fn = jax.jit(energy_fn)
force_fn = jax.jit(jax_md.quantity.force(energy_fn))

# Initialize the short and long-range neighbor lists.
nbrs = neighbor_fn.allocate(
    positions_init,
    box=box
)
nbrs_lr = neighbor_fn_lr.allocate(
    positions_init,
    box=box
)

/Users/thorbenfrank/Documents/venvs/sup-gems/lib/python3.9/site-packages/jax/_src/numpy/lax_numpy.py:3289: FutureWarning: None encountered in jnp.array(); this is currently treated as NaN. In the future this will result in an error.
  return array(a, dtype=dtype, copy=bool(copy), order=order)


In [5]:
# Check that the energy function is working.
energy_fn(
    positions_init, 
    neighbor=nbrs.idx, 
    neighbor_lr=nbrs_lr.idx, 
    box=box
)

Array(-180.43394, dtype=float32)

# Structure Relaxation

In [6]:
min_cycles = 5
min_steps = 10

fire_init, fire_apply = jax_md.minimize.fire_descent(
    energy_fn, 
    shift, 
    dt_start = 0.05, 
    dt_max = 0.1, 
    n_min = 2
)

fire_apply = jax.jit(fire_apply)
fire_state = fire_init(
    positions_init, 
    box=box, 
    neighbor=nbrs.idx,
    neighbor_lr=nbrs_lr.idx
)

@jax.jit
def step_fire_fn(i, fire_state):
    
    fire_state, nbrs, nbrs_lr = fire_state
    
    fire_state = fire_apply(
        fire_state, 
        neighbor=nbrs.idx, 
        neighbor_lr=nbrs_lr.idx, 
        box=box
    )
    
    nbrs = nbrs.update(
        fire_state.position,
        neighbor=nbrs.idx
    )
    
    nbrs_lr = nbrs_lr.update(
        fire_state.position,
        neighbor_lr=nbrs_lr.idx
    )
    
    return fire_state, nbrs, nbrs_lr

print('Step\tE\tFmax')
print('----------------------------------------')
for i in range(min_cycles):
    fire_state, nbrs, nbrs_lr = jax.lax.fori_loop(
        0, 
        min_steps, 
        step_fire_fn, 
        (fire_state, nbrs, nbrs_lr)
    )
    
    E = energy_fn(
        fire_state.position, 
        neighbor=nbrs.idx, 
        neighbor_lr=nbrs_lr.idx,
        box=box
    )

    F = force_fn(
        fire_state.position, 
        neighbor=nbrs.idx, 
        neighbor_lr=nbrs_lr.idx,
        box=box
    )
    
    print('{}\t{:.2f}\t{:.2f}'.format(i, E, np.abs(F).max()))
    

Step	E	Fmax
----------------------------------------
0	-180.91	0.51
1	-181.09	0.22
2	-181.20	0.16
3	-181.28	0.12
4	-181.32	0.06


In [7]:
# Print the delta between original and optimized positions for the first three atoms.
(positions_init - fire_state.position)[:3]

Array([[-0.06604004,  0.05084229,  0.06851196],
       [-0.00616455, -0.01034546,  0.04959106],
       [ 0.00949097,  0.0005188 , -0.03643799]], dtype=float32)

# Nose-Hoover Chain NVT Simulation

In [8]:
# Simulation parameters

timestep = 0.0005  # Time step in ps
nvt_cycles = 5  # Number of Cycles in the NPT.
nvt_steps = 50  # Number of NPT steps per cylce. The total number of MD steps equals npt_cylces * npt_steps

T_init = None  # Initial temperature.
T_nvt = 300  # Target temperature. 

chain = 3  # Number of chains in the Nose-Hoover chain.
chain_steps = 2  # Number of steps per chain.
sy_steps = 3
thermo = 100  # Thermo value in the Nose-Hoover chain. 

# Set the temprature at initialization
if T_init is None:
    T_init = float(T_nvt / nvt_cycles) 
else:
    T_init = T_init

# Dictionary with the NHC settings.
new_nhc_kwargs = {
    'chain_length': chain, 
    'chain_steps': chain_steps, 
    'sy_steps': sy_steps
}

# Convert to metal unit system.
unit = units.metal_unit_system()

timestep = timestep * unit['time']
T_init = T_init * unit['temperature']

rng_key = jax.random.PRNGKey(0)

/Users/thorbenfrank/Documents/venvs/sup-gems/lib/python3.9/site-packages/jax/_src/numpy/lax_numpy.py:166: UserWarning: Explicitly requested dtype float64 requested in asarray is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return asarray(x, dtype=self.dtype)


In [10]:
# Chosse Nose-Hoover thermostat.
init_fn, apply_fn = jax_md.simulate.nvt_nose_hoover(
    energy_fn, 
    shift, 
    dt=timestep, 
    kT=T_init,
    box=box,
    thermostat_kwargs=default_nhc_kwargs(thermo * timestep, new_nhc_kwargs)
)

apply_fn = jax.jit(apply_fn)
init_fn = jax.jit(init_fn)

# Initialize state using position and neigbhors structure relaxation.
state = init_fn(
    rng_key, 
    fire_state.position, 
    box=box, 
    neighbor=nbrs.idx, 
    neighbor_lr=nbrs_lr.idx,
    kT=T_init,
    mass=masses
)

@jax.jit
def step_nvt_fn(i, state):
    state, nbrs, nbrs_lr, box, temp_i = state
    
    state = apply_fn(
        state, 
        neighbor=nbrs.idx, 
        neighbor_lr=nbrs_lr.idx, 
        kT=temp_i,
        box=box
    )
    
    nbrs = nbrs.update(
        state.position, 
        neighbor=nbrs.idx, 
        box = box
    )
    
    nbrs_lr = nbrs_lr.update(
        state.position, 
        neighbor_lr=nbrs_lr.idx, 
        box = box
    )
    
    return state, nbrs, nbrs_lr, box, temp_i

# Track total time and step times averaged over cycle.
total_time = time.time()

positions_md = []

print('Step\tKE\tPE\tTotal Energy\tTemperature\tH\ttime/steps')
print('-----------------------------------------------------------------------------------')
for i in range(nvt_cycles):
    
    temp_i = T_nvt

    old_time = time.time()
    
    # Do `nvt_steps` NVT steps.
    new_state, nbrs, nbrs_lr, new_box, temp_i = jax.block_until_ready(
        jax.lax.fori_loop(
            0,
            nvt_steps,
            step_nvt_fn,
            (state, nbrs, nbrs_lr, box, temp_i)  # carry state is tuple
        )
    )
    
    new_time = time.time()
    
    # Check for overflor of both sr and lr neighbors.
    if nbrs.did_buffer_overflow:
        print('Neighbor list overflowed, reallocating.')
        nbrs = neighbor_fn.allocate(state.position, box = box)
        if nbrs_lr.did_buffer_overflow:
            print('Long-range neighbor list also overflowed, reallocating.')
            nbrs_lr = neighbor_fn_lr.allocate(state.position, box = box)
    elif nbrs_lr.did_buffer_overflow:
        print('Long-range neighbor list overflowed, reallocating.')
        nbrs_lr = neighbor_fn_lr.allocate(state.position, box = box)
    else:
        state = new_state
        box = new_box

    # Calculate some quantities for printing
    KE = jax_md.quantity.kinetic_energy(
        momentum=state.momentum,
        mass=state.mass
    )
    
    PE = energy_fn(
        state.position,
        neighbor=nbrs.idx,
        neighbor_lr=nbrs_lr.idx, 
        box=box
    )
    
    T = jax_md.quantity.temperature(
        momentum=state.momentum,
        mass = state.mass
    ) / unit['temperature']
    
    H = jax_md.simulate.nvt_nose_hoover_invariant(
        energy_fn, 
        state, 
        kT=temp_i,
        neighbor=nbrs.idx,
        neighbor_lr=nbrs_lr.idx, 
        box=box
    )

    positions_md.append(np.array(state.position))
    
    print(f'{i*nvt_steps}\t{KE:.2f}\t{PE:.2f}\t{KE+PE:.3f}\t{T:.1f}\t{H:.3f}\t{(new_time - old_time) / nvt_steps:.4f}')

print('Total_time: ', time.time()-total_time)

# Clear all caches
jax.clear_caches()

Step	KE	PE	Total Energy	Temperature	H	time/steps
-----------------------------------------------------------------------------------
0	0.31	-181.12	-180.813	42.6	-180.729	0.3449
50	0.39	-181.11	-180.721	54.4	-180.604	0.0157
100	0.47	-181.07	-180.601	65.3	-180.599	0.0157
150	0.58	-181.02	-180.439	80.3	-180.614	0.0158
200	0.74	-180.97	-180.230	101.6	-180.582	0.0157
Total_time:  20.574928998947144


# Visualization and IO

For now, we concatenated the positions to a simple python list `positions_md`. This allows to directly visualize the trajectory and write the frames to `xyz` after the simulation. However, for production runs it might be neccessary to save the frames (and potentially other information) along the way. For large structures, using `ase.io.write` can be very very slow and reduce the simulation time by sevaral order of magnitudes. To learn how to efficiently write frames and other statistics during simulation, check out the `production_run.ipynb` example notebooks. It uses `.hdf5` files to efficiently perform save operations.

In [11]:
# If you want to do visualization in the notebook, install nglview by doing `pip install nglview` in your virtualenv
import nglview as nv
from ase import Atoms

atoms_traj = []
for positions in positions_md:
    atoms_traj.append(
        Atoms(numbers=np.array(species), positions=positions),
    )

nv.show_asetraj(atoms_traj)

NGLWidget(max_frame=4)

In [ ]:
# Save the frames to xyz.
from ase.io import write

for frame in atoms_traj:
    write( 
        'nvt_md_trajectory.xyz',
        frame,
        append=True
    )